## RDD Transformations and Actions

In [2]:
from pyspark import SparkContext

In [7]:
sc = SparkContext()

25/01/02 15:37:51 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [47]:
ex2 = '../data/example2.txt'

In [48]:
sc.textFile(ex2)

../data/example2.txt MapPartitionsRDD[26] at textFile at NativeMethodAccessorImpl.java:0

In [49]:
text_rdd = sc.textFile(ex2)

In [32]:
words = text_rdd.map(lambda line: line.split())

In [33]:
words.collect()

[['first'],
 ['second', 'line'],
 ['then', 'third', 'line'],
 ['then', 'a', 'fourth', 'line', 'write', 'that', 'here']]

In [34]:
text_rdd.collect()

['first',
 'second line',
 'then third line',
 'then a fourth line write that here ']

In [36]:
flat_text = text_rdd.flatMap(lambda line: line.split()).collect

In [40]:
services = '../data/services.txt'

In [41]:
services = sc.textFile(services)

In [42]:
services.take(2)

['#EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00']

In [44]:
services.map(lambda x: x.split()).take(3)

[['#EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00']]

In [45]:
services.map(lambda x: x[1:] if x[0]=='#' else x).collect()

['EventId    Timestamp    Customer   State    ServiceID    Amount',
 '201       10/13/2017      100       NY       131          100.00',
 '204       10/18/2017      700       TX       129          450.00',
 '202       10/15/2017      203       CA       121          200.00',
 '206       10/19/2017      202       CA       131          500.00',
 '203       10/17/2017      101       NY       173          750.00',
 '205       10/19/2017      202       TX       121          200.00']

In [50]:
clean = services.map(lambda x: x[1:] if x[0]=='#' else x)

In [51]:
clean = clean.map(lambda x: x.split())

In [52]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [62]:
pairs = clean.map(lambda x: (x[3], x[-1]))

In [79]:
pairs.collect()

[('State', 'Amount'),
 ('NY', '100.00'),
 ('TX', '450.00'),
 ('CA', '200.00'),
 ('CA', '500.00'),
 ('NY', '750.00'),
 ('TX', '200.00')]

In [80]:
rekey = pairs.reduceByKey(lambda amt1,amt2: float(amt1) + float(amt2))

In [81]:
rekey.collect()

[('State', 'Amount'), ('NY', 850.0), ('TX', 650.0), ('CA', 700.0)]

In [68]:
clean.collect()

[['EventId', 'Timestamp', 'Customer', 'State', 'ServiceID', 'Amount'],
 ['201', '10/13/2017', '100', 'NY', '131', '100.00'],
 ['204', '10/18/2017', '700', 'TX', '129', '450.00'],
 ['202', '10/15/2017', '203', 'CA', '121', '200.00'],
 ['206', '10/19/2017', '202', 'CA', '131', '500.00'],
 ['203', '10/17/2017', '101', 'NY', '173', '750.00'],
 ['205', '10/19/2017', '202', 'TX', '121', '200.00']]

In [70]:
step1 = clean.map(lambda lst: (lst[3], lst[-1]))

In [71]:
step2 = step1.reduceByKey(lambda amt1, amt2: float(amt1) + float(amt2))

In [72]:
step3 = step2.filter(lambda x: not x[0]=='State')

In [73]:
step4 = step3.sortBy(lambda stAmount: stAmount[1], ascending=False)

In [74]:
step4.collect()

[('NY', 850.0), ('CA', 700.0), ('TX', 650.0)]

In [82]:
x = ['ID', 'State', 'Amount']

In [83]:
def func1(lst):
    return lst[-1]

In [ ]:
def func2(id_st_amt):
    (Id,st,amt) = id_st_amt
    return amt